# 04: Model Usage Demonstration

This notebook demonstrates loading the previously trained and serialized Maltese sentiment analysis pipeline (`naive_bayes_maltese_sentiment_analyzer.joblib`) and utilizing it for inference on raw text inputs. For successful deserialization and operation of the custom components within the pipeline, the accompanying `preprocessor.py` module, which contains all custom class definitions for the custom objoects within the pipeline, must be present in the Python import path.

The notebook then shows how to make predictions and retrieve sentiment scores.

Essential Python libraries (`os`, `sys`, `joblib`, `traceback`) are imported, along with the custom `preprocessor` module.

In [23]:
import os
import sys
import joblib
import traceback

import preprocessor # Needed to deserialize and use the model

The script then loads the serialized `deployable_pipeline` from the `naive_bayes_maltese_sentiment_analyzer.joblib` file.

In [24]:
# Loading the pipeline
MODEL_FILENAME = "naive_bayes_maltese_sentiment_analyzer.joblib"
loaded_model_pipeline = None

if os.path.exists(MODEL_FILENAME):
    print(f"Loading model from: {MODEL_FILENAME}")
    try:
        loaded_model_pipeline = joblib.load(MODEL_FILENAME)
        print("Model loaded successfully.")
    except Exception as e:
        print(f"CRITICAL ERROR loading the model: {e}")
        traceback.print_exc() 
        print(f"Ensure dependency `preprocessor.py` is loaded")
        sys.exit(1) # Exit if model cannot be loaded
else:
    print(f"Error: Model file '{MODEL_FILENAME}' not found.")
    print("Please ensure the model was saved correctly using the 'Train and Save' script.")
    sys.exit(1) # Exit if model file not found

Loading model from: naive_bayes_maltese_sentiment_analyzer.joblib
Model loaded successfully.


The `get_sentiment_prediction` function is defined to process raw text input using the loaded `loaded_model_pipeline`. It explicitly accesses the pipeline's named steps (`'custom_maltese_preprocessor'` and `'sentiment_model'`) to first transform the raw text and then generate a sentiment classification and associated probabilities from this preprocessed output. The input text, its preprocessed form, and the final sentiment prediction are printed. The function's usage is then demonstrated with a list of example sentences.

In [ ]:
# Example Usage
def get_sentiment_prediction(raw_text_input, pipeline):
    try:
        # Step 1: Access individual steps
        custom_preprocessor = pipeline.named_steps['custom_maltese_preprocessor']
        model = pipeline.named_steps['sentiment_model']
        
        # Step 2: Get intermediate preprocessed output
        intermediate_output = custom_preprocessor.transform([raw_text_input])
        preprocessed_text = intermediate_output.iloc[0]
        
        print(f"Input: '{raw_text_input}'")
        print(f"Preprocessed Text: '{preprocessed_text}'")

        # Step 3: Run sentiment model on intermediate output
        prediction_label = model.predict(intermediate_output)[0]
        probabilities = model.predict_proba(intermediate_output)[0]

        # Step 4: Print output
        print(f"Sentiment: {'Positive' if int(prediction_label) == 1 else 'Negative'} ({prediction_label})")
        print(f"Probability (Negative): {float(probabilities[0]):.4f}")
        print(f"Probability (Positive): {float(probabilities[1]):.4f}")

    except Exception as e:
        print(f"Error during prediction for '{raw_text_input}': {e}")


# Example Usage
sentences_to_predict_list = [
    "Dan il-film kien assolutament eccelenti!",
    "Dan is-servizz kien verament tajjeb u għoġobni ħafna!",
    "Xejn speċjali, ma tantx impressjonani.",
    "Xejn ma ghogobni",
    "This is an English sentence."
    "Tal misthija",
    "Qazziztni",
    "Ma nifilhekx",
    "Tal-Ostja Int u t team kollhu tieghek!",
    "Inkredibbli kif irnexxielhom jkisruha diska.",
    "Prosit ministru",
    "Inkredibbli", # Incorrectly labelled as Negative
    "Kilt ikel tajjeb", # Incorrectly labelled as Negative

    "nisrani", # Positive?
    "musulman", # Negative?
    ]

print("\nPredicting for a list of sentences:")
for sentence in sentences_to_predict_list:
    result = get_sentiment_prediction(sentence, loaded_model_pipeline)
    print()


Predicting for a list of sentences:
Input: 'Dan il-film kien assolutament eccelenti!'
Preprocessed Text: 'dan il film kien assolutament eccelenti !'
Sentiment: Negative (0)
Probability (Negative): 0.7144
Probability (Positive): 0.2856

Input: 'Dan is-servizz kien verament tajjeb u għoġobni ħafna!'
Preprocessed Text: 'dan is servizz kien verament tajjeb u għoġob ħaf !'
Sentiment: Positive (1)
Probability (Negative): 0.0098
Probability (Positive): 0.9902

Input: 'Xejn speċjali, ma tantx impressjonani.'
Preprocessed Text: 'xejn speċjali , ma tantx impressjonani .'
Sentiment: Negative (0)
Probability (Negative): 0.7801
Probability (Positive): 0.2199

Input: 'Xejn ma ghogobni'
Preprocessed Text: 'xejn ma ghogobni'
Sentiment: Negative (0)
Probability (Negative): 0.9720
Probability (Positive): 0.0280

Input: 'This is an English sentence.Tal misthija'
Preprocessed Text: 'this is an english sentence . tal mistħi'
Sentiment: Negative (0)
Probability (Negative): 0.9976
Probability (Positive): 0.